# Get stock market data & clean data

In [1]:
%matplotlib inline
import yfinance as yf
import quantstats as qs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# extend pandas functionality with metrics, etc.
qs.extend_pandas()

##

# Data collection

In [2]:
# Names of intersted stocks
# stock_names = {'ABBV', 'PFE', 'BAC', 'COKE',
#                'SQ', 'AAPL', 'AMD', 'FDX',
#                'EBAY', 'AMZN', 'C', 'F',
#                'GDX', 'USO'}

stock_names = {'AMZN'}

In [3]:
# Get a list of all closing prices
stock_list = {}

for stock in stock_names:
    
    stock_list[stock] = qs.utils.download_returns(stock)


    
# Limit all close prices to the last 5 years 
close_all = []

for name in stock_names:
    
    close_new = stock_list[name].loc["2016-01-01":]
    
    close_all.append(close_new)
    
    
# Convert close prices to dataframe
df_close = pd.DataFrame(close_all[:]).T
df_close.columns = stock_names

display(df_close)

,AMZN
Date,
2016-01-04,-0.057554
2016-01-05,-0.005024
2016-01-06,-0.001799
2016-01-07,-0.039058
2016-01-08,-0.001464
...,...
2021-08-20,0.003827
2021-08-23,0.020600
2021-08-24,0.012220


##

# Data Cleaning #1

In [4]:
# Drop duplicates
df_close.drop_duplicates(inplace=False)

# Drop NA and infinite values
df_close = df_close.replace(-np.inf, np.nan).dropna()
df_close.head()

,AMZN
Date,
2016-01-04,-0.057554
2016-01-05,-0.005024
2016-01-06,-0.001799
2016-01-07,-0.039058
2016-01-08,-0.001464


In [5]:
# Count nulls/checking final df
df_close.isnull().sum()

AMZN    0
dtype: int64

In [6]:
# Define time period to predict
pred_period = 1

# Compute the pct_change for 1 min 
df_returns = df_close.pct_change(pred_period)

df_returns.head()

,AMZN
Date,
2016-01-04,NaN
2016-01-05,-0.912714
2016-01-06,-0.641967
2016-01-07,20.715398
2016-01-08,-0.962518


In [7]:
# Shift the returns to convert them to forward returns i.e,. today's return = return from 2-days ahead
df_returns_forward = df_returns.shift(-1)

# Preview the DataFrame
df_returns_forward.head()

,AMZN
Date,
2016-01-04,-0.912714
2016-01-05,-0.641967
2016-01-06,20.715398
2016-01-07,-0.962518
2016-01-08,-13.028651


In [8]:
# Place stock data on top of each other along the length of the df
df_returns = pd.DataFrame(df_returns.unstack())

# Rename the column to returns
name = f'{pred_period}_Day_returns'

df_returns.rename(columns={0: name},
                  inplace = True)

# Reset the index 
df_returns.reset_index(inplace=True)

In [9]:
df_returns.head()

,level_0,Date,1_Day_returns
0,AMZN,2016-01-04,NaN
1,AMZN,2016-01-05,-0.912714
2,AMZN,2016-01-06,-0.641967
3,AMZN,2016-01-07,20.715398
4,AMZN,2016-01-08,-0.962518


##

# Shift data to create future close prices

In [10]:
# Create 5 and 10-day periods to predict
periods_to_pred = [5, 10]

for i in periods_to_pred:   
    
    # Calc percentage change for each time period
    returns_temp = df_close.pct_change(i)
    
    # Stack returns  
    returns_temp = pd.DataFrame(returns_temp.unstack())
    
    # Rename column 
    name = f'{i}_Day_returns'
    returns_temp.rename(columns={0:name},
                        inplace=True)
    
    # Reset index 
    returns_temp.reset_index(inplace=True)
    
    # Merge df 
    df_returns = pd.merge(df_returns,
                          returns_temp,
                          left_on=['level_0', 'Date'],
                          right_on=['level_0', 'Date'],
                          how='left', 
                          suffixes=('_original', 'right'))

In [11]:
# Check data
df_returns.tail(20)

,level_0,Date,1_Day_returns,5_Day_returns,10_Day_returns
1403,AMZN,2021-07-30,8.033824,-15.788546,3.771510
1404,AMZN,2021-08-02,-1.015453,-0.901007,-1.173775
1405,AMZN,2021-08-03,7.925548,-1.525713,0.569320
1406,AMZN,2021-08-04,-1.327994,-4.157699,-2.018171
1407,AMZN,2021-08-05,-2.852693,-1.757148,-0.569727
1408,AMZN,2021-08-06,-2.450606,-0.878421,-2.797977
1409,AMZN,2021-08-09,-0.900215,-1.785086,-1.077718
1410,AMZN,2021-08-10,5.909077,-1.607719,-0.680514
1411,AMZN,2021-08-11,0.356861,1.514041,-8.938584
1412,AMZN,2021-08-12,-1.402128,-0.454325,-1.413156


In [12]:
# Prepare df for ML classification models
ml_df_returns = df_returns.copy()

ml_df_returns["1_Day_binary"] = np.where(ml_df_returns["1_Day_returns"] >= 0, 1, 0)
ml_df_returns["5_Day_binary"] = np.where(ml_df_returns["5_Day_returns"] >= 0, 1, 0)
ml_df_returns["10_Day_binary"] = np.where(ml_df_returns["10_Day_returns"] >= 0, 1, 0)

ml_df_returns.head(10)

,level_0,Date,1_Day_returns,5_Day_returns,10_Day_returns,1_Day_binary,5_Day_binary,10_Day_binary
0,AMZN,2016-01-04,NaN,NaN,NaN,0,0,0
1,AMZN,2016-01-05,-0.912714,NaN,NaN,0,0,0
2,AMZN,2016-01-06,-0.641967,NaN,NaN,0,0,0
3,AMZN,2016-01-07,20.715398,NaN,NaN,1,0,0
4,AMZN,2016-01-08,-0.962518,NaN,NaN,0,0,0
5,AMZN,2016-01-11,-13.028651,-1.305970,NaN,0,0,0
6,AMZN,2016-01-12,-0.986209,-1.048343,NaN,0,0,0
7,AMZN,2016-01-13,-241.435912,31.464876,NaN,0,1,0
8,AMZN,2016-01-14,-1.329377,-1.492424,NaN,0,0,0
9,AMZN,2016-01-15,-3.000839,25.286009,NaN,0,1,0


##

# Data Cleaning #2

In [13]:
# Drop na
df_returns.dropna(axis=0, 
                  how="any",
                  inplace=True)

# Create a multi index based on stock name and time
df_returns.set_index(['level_0', 'Date'],
                  inplace=True)

df_returns.head()

1_Day_returns  5_Day_returns  10_Day_returns
level_0 Date                                                    
AMZN    2016-01-19      -1.195972      -0.571746       -1.131033
        2016-01-20      -1.625508     -20.423698       -0.060992
        2016-01-21      -2.204963      -1.097343       -4.160241
        2016-01-22       5.535157       0.931386       -1.951061
        2016-01-25      -0.993228      -1.006537       -1.171831

In [14]:
#Drop duplicates
df_returns.drop_duplicates(inplace=False)

#Drop Null and inf
df_returns=df_returns.replace([np.inf, -np.inf], np.nan).dropna(axis=0)

In [15]:
# Drop na
ml_df_returns.dropna(axis=0, 
                     how="any",
                     inplace=True)

# Create a multi index based on stock name and time
ml_df_returns.set_index(['level_0', 'Date'],
                        inplace=True)

ml_df_returns.head()

1_Day_returns  5_Day_returns  10_Day_returns  \
level_0 Date                                                       
AMZN    2016-01-19      -1.195972      -0.571746       -1.131033   
        2016-01-20      -1.625508     -20.423698       -0.060992   
        2016-01-21      -2.204963      -1.097343       -4.160241   
        2016-01-22       5.535157       0.931386       -1.951061   
        2016-01-25      -0.993228      -1.006537       -1.171831   

                    1_Day_binary  5_Day_binary  10_Day_binary  
level_0 Date                                                   
AMZN    2016-01-19             0             0              0  
        2016-01-20             0             0              0  
        2016-01-21             0             0              0  
        2016-01-22             1             1              0  
        2016-01-25             0             0              0

In [16]:
#Drop duplicates
ml_df_returns.drop_duplicates(inplace=False)

#Drop Null and inf
ml_df_returns=ml_df_returns.replace([np.inf, -np.inf], np.nan).dropna(axis=0)

##

# Final check

In [17]:
# Count nulls/Checking final df
df_returns.isnull().sum()

1_Day_returns     0
5_Day_returns     0
10_Day_returns    0
dtype: int64

In [18]:
df_returns.describe()

,1_Day_returns,5_Day_returns,10_Day_returns
count,1410.000000,1410.000000,1410.000000
mean,0.225065,-1.759446,-1.971092
std,41.937131,33.032745,25.319872
min,-526.541898,-1030.604071,-517.903721
25%,-1.915940,-1.962995,-2.061186
50%,-0.947609,-0.995155,-1.040724
75%,0.122761,0.047078,-0.016457
max,1068.227704,447.408560,215.846375


In [19]:
ml_df_returns.isnull().sum()

1_Day_returns     0
5_Day_returns     0
10_Day_returns    0
1_Day_binary      0
5_Day_binary      0
10_Day_binary     0
dtype: int64

In [20]:
ml_df_returns.describe()

,1_Day_returns,5_Day_returns,10_Day_returns,1_Day_binary,5_Day_binary,10_Day_binary
count,1410.000000,1410.000000,1410.000000,1410.000000,1410.000000,1410.000000
mean,0.225065,-1.759446,-1.971092,0.265957,0.256738,0.247518
std,41.937131,33.032745,25.319872,0.441998,0.436988,0.431723
min,-526.541898,-1030.604071,-517.903721,0.000000,0.000000,0.000000
25%,-1.915940,-1.962995,-2.061186,0.000000,0.000000,0.000000
50%,-0.947609,-0.995155,-1.040724,0.000000,0.000000,0.000000
75%,0.122761,0.047078,-0.016457,1.000000,1.000000,0.000000
max,1068.227704,447.408560,215.846375,1.000000,1.000000,1.000000


In [21]:
# Save the dataframe as a csv file
df_returns.to_csv("Returns_ForTimeSeries.csv")
ml_df_returns.to_csv("Returns_ForML_Classification.csv")